In [1]:
import os
from pathlib import Path
import numpy as np
import logging

import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u

from gammapy.catalog import CATALOG_REGISTRY
from gammapy.modeling.models import Models
from gammapy.maps import Map

In [101]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig

from asgardpy.data.dataset_3d import Dataset3DGeneration

from asgardpy.data.target import (
    set_models, 
    apply_selection_mask_to_models,
    read_models_from_asgardpy_config
)

In [102]:
os.environ['GAMMAPY_DATA']

'/home/chaitanya/software/gammapy-datasets/1.1/'

In [103]:
log = logging.getLogger("test filtering of Models")

In [104]:
config_file = "../src/asgardpy/tests/config_test_gadf.yaml"

In [105]:
config_main = AsgardpyConfig().read(config_file)

# Update path of test data

In [106]:
config_main.dataset3d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}hess-dl3-dr1/"
config_main.dataset3d.instruments[0].dataset_info.background.exclusion.exclusion_file = (
    f"{os.environ['GAMMAPY_DATA']}joint-crab/exclusion/exclusion_mask_crab.fits.gz"
)
config_main.dataset1d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}magic/rad_max/data/"

In [107]:
config_main.target.use_catalog.selection_radius = "10 deg"
config_main.target.roi_selection.roi_radius = "2.8 deg"

In [108]:
config_main.general.dict()

{'log': {'level': 'info',
  'filename': '',
  'filemode': 'w',
  'format': '',
  'datefmt': ''},
 'outdir': PosixPath('.'),
 'n_jobs': 10,
 'parallel_backend': 'multiprocessing',
 'steps': ['datasets-3d', 'datasets-1d', 'fit', 'flux-points'],
 'overwrite': True,
 'stacked_dataset': True}

In [109]:
config_main.target.dict()

{'source_name': 'Crab Nebula',
 'sky_position': {'frame': 'icrs',
  'lon': <Quantity 83.633 deg>,
  'lat': <Quantity 22.014 deg>,
  'radius': <Quantity 0. deg>},
 'use_uniform_position': True,
 'models_file': PosixPath('.'),
 'datasets_with_fov_bkg_model': ['HESS'],
 'use_catalog': {'name': '3fhl',
  'selection_radius': <Quantity 10. deg>,
  'exclusion_radius': <Quantity 0.2 deg>},
 'components': [{'name': 'Crab Nebula',
   'type': 'SkyModel',
   'datasets_names': [''],
   'spectral': {'type': 'LogParabolaSpectralModel',
    'parameters': [{'name': 'amplitude',
      'value': 1e-06,
      'unit': 'cm-2 s-1 TeV-1',
      'error': 1.5e-07,
      'min': 1e-13,
      'max': 0.01,
      'frozen': False},
     {'name': 'reference',
      'value': 0.01,
      'unit': 'TeV',
      'error': 0.0,
      'min': 0.0001,
      'max': 100.0,
      'frozen': True},
     {'name': 'alpha',
      'value': 1.7,
      'unit': '',
      'error': 0.1,
      'min': 0.5,
      'max': 5.0,
      'frozen': False

In [110]:
config_main.dataset3d.dict()

{'type': '3d',
 'instruments': [{'name': 'HESS',
   'input_dl3': [{'type': 'gadf-dl3',
     'input_dir': PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1'),
     'glob_pattern': {'dl3': 'data/hess_*fits.gz'}}],
   'input_dl4': False,
   'dataset_info': {'name': 'HESS',
    'key': [],
    'observation': {'obs_ids': [],
     'obs_file': PosixPath('.'),
     'obs_time': [{'format': 'iso',
       'start': '2004-12-04 22:00:00.000',
       'stop': '2004-12-04 22:44:00.000'},
      {'format': 'mjd', 'start': '53343.947916666', 'stop': '53343.989583333'},
      {'format': 'iso',
       'start': '2004-12-05 00:00:00.000',
       'stop': '2004-12-07 00:00:00.000'}],
     'obs_cone': {'frame': 'icrs',
      'lon': <Quantity 83.633 deg>,
      'lat': <Quantity 22.014 deg>,
      'radius': <Quantity 5. deg>},
     'required_irfs': ['aeff', 'edisp', 'psf', 'bkg']},
    'map_selection': ['counts', 'exposure', 'background', 'psf', 'edisp'],
    'geom': {'wcs': {'skydir': {'frame'

In [52]:
config_main.dataset1d.dict()

{'type': '1d',
 'instruments': [{'name': 'MAGIC',
   'input_dl3': [{'type': 'gadf-dl3',
     'input_dir': PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/magic/rad_max/data'),
     'glob_pattern': {'dl3': '2013*fits'}}],
   'input_dl4': False,
   'dataset_info': {'name': 'MAGIC',
    'geom': {'wcs': {'skydir': {'frame': 'icrs',
       'lon': <Quantity 0. deg>,
       'lat': <Quantity 0. deg>,
       'radius': <Quantity 0. deg>},
      'binsize': <Quantity 0.02 deg>,
      'proj': 'TAN',
      'map_frame_shape': {'width': <Quantity 6. deg>,
       'height': <Quantity 6. deg>},
      'binsize_irf': <Quantity 0.2 deg>},
     'selection': {'offset_max': <Quantity 2.5 deg>},
     'axes': [{'name': 'energy',
       'axis': {'min': <Quantity 0.05 TeV>,
        'max': <Quantity 10. TeV>,
        'nbins': 5,
        'per_decade': True},
       'axis_custom': {'edges': [], 'unit': 'TeV'}},
      {'name': 'energy_true',
       'axis': {'min': <Quantity 0.01 TeV>,
        'max': <Quantity

In [111]:
analysis = AsgardpyAnalysis(config_main)

Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}


In [112]:
instruments_list = config_main.dataset3d.instruments
print(len(instruments_list), "number of 3D dataset information provided")

config_3d_dataset = instruments_list[0]
print(f"Instrument selected is {config_3d_dataset.name}")

1 number of 3D dataset information provided
Instrument selected is HESS


In [113]:
key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

The list of different keys or modes of observations for the selected instrument are []


In [114]:
%%time
generate_3d_dataset = Dataset3DGeneration(
    log, config_3d_dataset, analysis.config
)

CPU times: user 59 µs, sys: 23 µs, total: 82 µs
Wall time: 83.4 µs


In [91]:
print(generate_3d_dataset.exclusion_mask)

None


In [92]:
print(generate_3d_dataset.diffuse_models)

{'gal_diffuse': None, 'iso_diffuse': None, 'key_name': None, 'gal_diffuse_cutout': None}


In [93]:
print(generate_3d_dataset.list_source_models)

[]


In [94]:
exclusion_regions = []

In [95]:
filled_skymodel = False

In [115]:
%%time
file_list = generate_3d_dataset.read_to_objects(key_names)

CPU times: user 1.96 ms, sys: 760 µs, total: 2.72 ms
Wall time: 2.03 ms


In [116]:
file_list

[PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020136.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020137.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020151.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020275.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020282.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020283.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020301.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1_obs_id_020302.fits.gz'),
 PosixPath('/home/chaitanya/software/gammapy-datasets/1.1/hess-dl3-dr1/data/hess_dl3_dr1

# Start filtering data/models

## Filter Observations

In [117]:
from asgardpy.base.geom import (
    get_source_position, 
    generate_geom,
)
from asgardpy.base.reduction import (
    get_filtered_observations,
)

In [67]:
observations = get_filtered_observations(
    dl3_path=generate_3d_dataset.config_3d_dataset.input_dl3[0].input_dir,
    obs_config=generate_3d_dataset.config_3d_dataset.dataset_info.observation,
    log=generate_3d_dataset.log,
)

Observation ID list selected: 23523 23526 23559
Observations selected: 3 out of 3.


In [68]:
print(observations)

Observations
Number of observations: 3
Observation

	obs id            : 23523 
 	tstart            : 53343.92
	tstop             : 53343.94
	duration          : 1687.00 s
	pointing (icrs)   : 83.6 deg, 21.5 deg

	deadtime fraction : 6.2%
Observation

	obs id            : 23526 
 	tstart            : 53343.95
	tstop             : 53343.97
	duration          : 1683.00 s
	pointing (icrs)   : 83.6 deg, 22.5 deg

	deadtime fraction : 6.6%
Observation

	obs id            : 23559 
 	tstart            : 53345.96
	tstop             : 53345.98
	duration          : 1686.00 s
	pointing (icrs)   : 85.3 deg, 22.0 deg

	deadtime fraction : 6.4%



In [ ]:
# filter observations based on the obs cone

In [118]:
generate_3d_dataset.config_3d_dataset.dataset_info.observation.dict()

{'obs_ids': [],
 'obs_file': PosixPath('.'),
 'obs_time': [{'format': 'iso',
   'start': '2004-12-04 22:00:00.000',
   'stop': '2004-12-04 22:44:00.000'},
  {'format': 'mjd', 'start': '53343.947916666', 'stop': '53343.989583333'},
  {'format': 'iso',
   'start': '2004-12-05 00:00:00.000',
   'stop': '2004-12-07 00:00:00.000'}],
 'obs_cone': {'frame': 'icrs',
  'lon': <Quantity 83.633 deg>,
  'lat': <Quantity 22.014 deg>,
  'radius': <Quantity 5. deg>},
 'required_irfs': ['aeff', 'edisp', 'psf', 'bkg']}

In [119]:
temp_obs_1 = generate_3d_dataset.config_3d_dataset.dataset_info.observation
temp_obs_1.obs_cone = {}

In [120]:
temp_obs_1.dict()

{'obs_ids': [],
 'obs_file': PosixPath('.'),
 'obs_time': [{'format': 'iso',
   'start': '2004-12-04 22:00:00.000',
   'stop': '2004-12-04 22:44:00.000'},
  {'format': 'mjd', 'start': '53343.947916666', 'stop': '53343.989583333'},
  {'format': 'iso',
   'start': '2004-12-05 00:00:00.000',
   'stop': '2004-12-07 00:00:00.000'}],
 'obs_cone': {'frame': 'icrs',
  'lon': <Quantity 0. deg>,
  'lat': <Quantity 0. deg>,
  'radius': <Quantity 0. deg>},
 'required_irfs': ['aeff', 'edisp', 'psf', 'bkg']}

In [121]:
obs_temp_1 = get_filtered_observations(
    dl3_path=generate_3d_dataset.config_3d_dataset.input_dl3[0].input_dir,
    obs_config=temp_obs_1,
    log=generate_3d_dataset.log,
)

Observation ID list selected: 23523 23526 23559
Observations selected: 3 out of 3.


In [122]:
print(obs_temp_1.ids)

['23523', '23526', '23559']


In [123]:
temp_obs_2 = generate_3d_dataset.config_3d_dataset.dataset_info.observation
print(temp_obs_2.obs_cone)

frame='icrs' lon=<Quantity 0. deg> lat=<Quantity 0. deg> radius=<Quantity 0. deg>


In [124]:
generate_3d_dataset.config_3d_dataset.dataset_info.observation.dict()

{'obs_ids': [],
 'obs_file': PosixPath('.'),
 'obs_time': [{'format': 'iso',
   'start': '2004-12-04 22:00:00.000',
   'stop': '2004-12-04 22:44:00.000'},
  {'format': 'mjd', 'start': '53343.947916666', 'stop': '53343.989583333'},
  {'format': 'iso',
   'start': '2004-12-05 00:00:00.000',
   'stop': '2004-12-07 00:00:00.000'}],
 'obs_cone': {'frame': 'icrs',
  'lon': <Quantity 0. deg>,
  'lat': <Quantity 0. deg>,
  'radius': <Quantity 0. deg>},
 'required_irfs': ['aeff', 'edisp', 'psf', 'bkg']}

In [ ]:
obs_temp_2 = get_filtered_observations(
    dl3_path=generate_3d_dataset.config_3d_dataset.input_dl3[0].input_dir,
    obs_config=temp_obs,
    log=generate_3d_dataset.log,
)